In [ ]:
!apt-get update #update linux
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #download and install openjdk
!wget -q http://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz #download spark binary (gunzip). -q: Turn off Wget’s output.
!tar xf spark-3.2.1-bin-hadoop2.7.tgz #extract the spark package
!pip install -q findspark #install the findspark package

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,167 kB]
Hit:14 http://ppa.lau

In [ ]:
!ls /usr/lib/jvm/

default-java  java-1.11.0-openjdk-amd64  java-11-openjdk-amd64


In [ ]:
!ls /usr/lib/jvm/java-8-openjdk-amd64

ls: cannot access '/usr/lib/jvm/java-8-openjdk-amd64': No such file or directory


In [ ]:
#set environment variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [ ]:
!ls #check directory

sample_data


In [ ]:
!ls spark-3.2.1-bin-hadoop2.7

bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin


In [ ]:
!ls spark-3.2.1-bin-hadoop2.7/bin/

beeline		      pyspark		spark-class.cmd   spark-sql
beeline.cmd	      pyspark2.cmd	sparkR		  spark-sql2.cmd
docker-image-tool.sh  pyspark.cmd	sparkR2.cmd	  spark-sql.cmd
find-spark-home       run-example	sparkR.cmd	  spark-submit
find-spark-home.cmd   run-example.cmd	spark-shell	  spark-submit2.cmd
load-spark-env.cmd    spark-class	spark-shell2.cmd  spark-submit.cmd
load-spark-env.sh     spark-class2.cmd	spark-shell.cmd


In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
# import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 

In [ ]:
path='spark-3.2.1-bin-hadoop2.7/README.md'
textFile = spark.read.text(path)

In [ ]:
type(textFile)

pyspark.sql.dataframe.DataFrame

In [ ]:
textFile.count() # Number of rows in this DataFrame

109

In [ ]:
textFile.first()  # First row in this DataFrame

Row(value='# Apache Spark')

In [ ]:
textFile.filter(textFile.value.contains("Spark")).count()  # How many lines contain "Spark"?

19

In [ ]:
type(textFile.filter(textFile.value.contains("Spark")))

pyspark.sql.dataframe.DataFrame

In [ ]:
textFile.filter(textFile.value.contains("Spark")).show(truncate=False) #truncate=False will show the full content

+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                        |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|# Apache Spark                                                                                                                                               |
|Spark is a unified analytics engine for large-scale data processing. It provides                                                                             |
|rich set of higher-level tools including Spark SQL for SQL and DataFrames,                                                                                   |
|[![PySpark Coverage](https://codecov.io

In [ ]:
# find the max number of words in a line
from pyspark.sql.functions import *
textFile.select(size(split(textFile.value, "\s+")).name("numWords")).agg(max(col("numWords"))).collect()

[Row(max(numWords)=16)]

In [ ]:
type(textFile.select(size(split(textFile.value, "\s+")).name("numWords")).agg(max(col("numWords"))))

pyspark.sql.dataframe.DataFrame

In [ ]:
textFile2 = sc.textFile(path) #import txt to RDD
type(textFile2)

pyspark.rdd.RDD

In [ ]:
textFile2.take(5) #for RDD, use 'take' to get the first few rows

['# Apache Spark',
 '',
 'Spark is a unified analytics engine for large-scale data processing. It provides',
 'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 'supports general computation graphs for data analysis. It also supports a']

In [ ]:
# find the max number of words in a line using map reduce
textFile2.map(lambda line : len(line.split(" "))).reduce(lambda a, b : a if (a > b) else b)

16